## Concatenação e filtragem dos demais arquivos que compõem o dataset do ENADE

- Filtragem com base nos arquivos gerados em enade_filtrado.csv

Objetivo: obter as respostas linha a linha de cada aluno dos cursos de computação cadastrados na base do ENADE e que foram extraídos na etapa anterior.

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Caminhos manuais para cada ano
microdados_2014 = '/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2014_filtrado.csv'
microdados_2017 = '//content/drive/MyDrive/EDM 2025.1/ENADE/enade_2017_filtrado.csv'
microdados_2021 = '/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2021_filtrado.csv'

In [ ]:
cursos2014 = pd.read_csv(microdados_2014, sep=';', encoding='latin1', low_memory=False)
cursos2017 = pd.read_csv(microdados_2017, sep=';', encoding='latin1', low_memory=False)
cursos2021 = pd.read_csv(microdados_2021, sep=';', encoding='latin1', low_memory=False)

In [ ]:
cursos2014

,NU_ANO,CO_CURSO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO
0,2014,100075,71,2,10028,4004,1,2411205,24,2
1,2014,100105,71,2,10028,4004,1,2408300,24,2
2,2014,100244,1082,1,10026,72,1,2408102,24,2
3,2014,100246,1082,1,10026,79,1,2408102,24,2
4,2014,101830,587,1,10028,4006,1,2613909,26,2
...,...,...,...,...,...,...,...,...,...,...
234,2014,98984,579,1,10028,4005,1,2512903,25,2
235,2014,99202,2242,5,10022,4004,1,2611606,26,2
236,2014,99300,583,1,10028,5809,1,2312908,23,2
237,2014,99406,3,1,10028,4006,1,2806701,28,2


### Filtragem dos arquivos .txt de cada ano:

Nesta etapa, é feita uma extração dos arquivos .txt de cada ano que se tornaram interessantes para o projeto, onde é lido e extraído apenas as respostas de pessoas com base na coluna **CO_CURSO**.

In [ ]:
#arq2: Edição, código de curso e informações acadêmicas dos estudantes
#arq3: nº de itens válidos por seção da prova, vetores, tipos de presença, tipos de situação das questões, notas e respostas do Questionário de Percepção

In [ ]:
import pandas as pd

def processar_enade_por_curso(
    base_filtrada, pasta_ano, edicao_enade, lista_numeros_arquivos, lista_atributos
):
    # 1. Extrai lista única de códigos de curso válidos (TI)
    cursos_ti = pd.read_csv(base_filtrada, sep=';', encoding='latin1', dtype=str)
    cursos_ti = cursos_ti[cursos_ti["CO_CURSO"] != "DJ1"]
    cursos_ti["CO_CURSO"] = cursos_ti["CO_CURSO"].astype(int)
    codigos_ti = cursos_ti["CO_CURSO"].unique().tolist()

    # 2. Lista para armazenar os DataFrames filtrados e ordenados
    lista_df_arquivos = []

    for numero in lista_numeros_arquivos:
        caminho = f'/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/{pasta_ano}/microdados{edicao_enade}_arq{numero}.txt'

        usecols = ['CO_CURSO', 'NT_GER', 'TP_PRES'] if numero == 3 else None
        df = pd.read_csv(caminho, sep=';', encoding='latin1', dtype=str, usecols=usecols, low_memory=False)

        df = df[df["CO_CURSO"] != "DJ1"]
        df["CO_CURSO"] = df["CO_CURSO"].astype(int)

        # Filtra apenas os cursos da área de TI
        df = df[df["CO_CURSO"].isin(codigos_ti)]

        # Ordena por CO_CURSO para manter alinhamento entre os arquivos
        df.sort_values("CO_CURSO", inplace=True)
        df.reset_index(drop=True, inplace=True)

        lista_df_arquivos.append(df)

    df_final = pd.concat(lista_df_arquivos, axis=1)

    df_final = df_final.loc[:, ~df_final.columns.duplicated()]

    df_final = df_final[lista_atributos]

    caminho_saida = f'/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/{pasta_ano}/computacao_enade_{edicao_enade}.csv'
    df_final.to_csv(caminho_saida, index=False, sep=';')
    print(f"✅ Arquivo salvo com sucesso: {caminho_saida}")


In [ ]:
ano = 2014
pasta_ano = f'/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/{ano}'
caminho_base_ti = '/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2014_filtrado.csv'

lista_colunas = [
    "NU_ANO", "CO_CURSO", "CO_GRUPO", "CO_UF_CURSO", "CO_REGIAO_CURSO",
    "CO_CATEGAD", "CO_ORGACAD", "IN_MATUT", "IN_VESPER", "IN_NOTURNO", "TP_PRES",
    "NT_GER", "TP_SEXO", "NU_IDADE", "ANO_FIM_EM", "ANO_IN_GRAD",
    "QE_I01", "QE_I02", "QE_I03", "QE_I04", "QE_I05",
    "QE_I08", "QE_I09", "QE_I10", "QE_I11", "QE_I12", "QE_I13",
    "QE_I15", "QE_I17", "QE_I21"
]

#lê cursos de TI do arquivo de base filtrada extraído anteriormente
cursos_ti = pd.read_csv(caminho_base_ti, sep=';', encoding='latin1', dtype=str)
cursos_ti = cursos_ti[cursos_ti["CO_CURSO"] != "DJ1"]
cursos_ti["CO_CURSO"] = cursos_ti["CO_CURSO"].astype(int)
codigos_ti = cursos_ti["CO_CURSO"].unique().tolist()

arquivos_txt = sorted([
    f for f in os.listdir(pasta_ano)
    if f.endswith('.txt') and f.startswith(f'microdados{ano}_arq')
])

#verificação de dados ausentes em cada arquivo em cada col especifica que escolhi
for nome_arquivo in arquivos_txt:
    caminho = os.path.join(pasta_ano, nome_arquivo)

    try:
        df = pd.read_csv(caminho, sep=';', encoding='latin1', dtype=str, usecols=lambda c: c in lista_colunas, low_memory=False)

        if 'CO_CURSO' not in df.columns:
            print(f"⚠️ {nome_arquivo} não possui a coluna 'CO_CURSO'. Pulando.")
            continue

        df = df[df["CO_CURSO"] != "DJ1"]
        df["CO_CURSO"] = df["CO_CURSO"].astype(int)
        df = df[df["CO_CURSO"].isin(codigos_ti)]

        # Verifica dados ausentes (NaN + strings vazias)
        colunas_validas = [col for col in lista_colunas if col in df.columns]
        ausentes = df[colunas_validas].isnull().sum() + (df[colunas_validas] == '').sum()

        print(f"\n📄 Arquivo: {nome_arquivo} — Cursos de TI\n")
        print(ausentes[ausentes > 0].sort_values(ascending=False))

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")


/tmp/ipython-input-5-319341767.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_CURSO"] = df["CO_CURSO"].astype(int)



📄 Arquivo: microdados2014_arq1.txt — Cursos de TI

Series([], dtype: int64)

📄 Arquivo: microdados2014_arq10.txt — Cursos de TI

QE_I04    1114
dtype: int64


/tmp/ipython-input-5-319341767.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_CURSO"] = df["CO_CURSO"].astype(int)



📄 Arquivo: microdados2014_arq11.txt — Cursos de TI

QE_I05    1114
dtype: int64

📄 Arquivo: microdados2014_arq14.txt — Cursos de TI

QE_I08    1114
dtype: int64

📄 Arquivo: microdados2014_arq15.txt — Cursos de TI

QE_I09    1114
dtype: int64

📄 Arquivo: microdados2014_arq16.txt — Cursos de TI

QE_I10    1114
dtype: int64


/tmp/ipython-input-5-319341767.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_CURSO"] = df["CO_CURSO"].astype(int)


...


📄 Arquivo: microdados2014_arq3.txt — Cursos de TI

NT_GER    1620
dtype: int64

📄 Arquivo: microdados2014_arq5.txt — Cursos de TI

TP_SEXO    2
dtype: int64


/tmp/ipython-input-5-319341767.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_CURSO"] = df["CO_CURSO"].astype(int)



📄 Arquivo: microdados2014_arq6.txt — Cursos de TI

NU_IDADE    2
dtype: int64


/tmp/ipython-input-5-319341767.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_CURSO"] = df["CO_CURSO"].astype(int)



📄 Arquivo: microdados2014_arq7.txt — Cursos de TI

QE_I01    1114
dtype: int64

📄 Arquivo: microdados2014_arq8.txt — Cursos de TI

QE_I02    1114
dtype: int64

📄 Arquivo: microdados2014_arq9.txt — Cursos de TI

QE_I03    1114
dtype: int64


### 2014

In [ ]:
# 2014
processar_enade_por_curso(
    base_filtrada='/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2014_filtrado.csv',
    pasta_ano='2014',
    edicao_enade=2014,
    lista_numeros_arquivos=[1,2, 3, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 23, 27],
    lista_atributos=[
        "NU_ANO", "CO_CURSO", "CO_GRUPO", "CO_UF_CURSO", "CO_REGIAO_CURSO",
        "CO_CATEGAD", "CO_ORGACAD", "IN_MATUT","IN_VESPER", "IN_NOTURNO", "TP_PRES",
        'NT_GER', 'TP_SEXO', 'NU_IDADE', 'ANO_FIM_EM', 'ANO_IN_GRAD',
        'QE_I01', 'QE_I02', 'QE_I03',
        'QE_I04', 'QE_I05', 'QE_I08', 'QE_I09', 'QE_I10', 'QE_I11',
        'QE_I12', 'QE_I13', 'QE_I15', 'QE_I17', 'QE_I21'
    ]
)

✅ Arquivo salvo com sucesso: /content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2014/computacao_enade_2014.csv


In [ ]:
computacao_resp2014 = pd.read_csv('/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2014/computacao_enade_2014.csv', sep=';')

computacao_resp2014[computacao_resp2014['CO_CURSO'] == 118420].shape[0]

ausentes_qe_i17 = computacao_resp2014['QE_I17'].isna().sum() + (computacao_resp2014['QE_I17'] == '').sum()

print(f"Total de valores ausentes em QE_I17: {ausentes_qe_i17}")


Total de valores ausentes em QE_I17: 1115


In [ ]:
computacao_resp2014.isnull().sum()

,0
NU_ANO,0
CO_CURSO,0
CO_GRUPO,0
CO_UF_CURSO,0
CO_REGIAO_CURSO,0
CO_CATEGAD,0
CO_ORGACAD,0
IN_MATUT,2
IN_VESPER,2
IN_NOTURNO,2


In [ ]:
sorted(computacao_resp2014['ANO_FIM_EM'].unique())

[np.float64(1969.0),
 np.float64(1972.0),
 np.float64(1974.0),
 np.float64(1975.0),
 np.float64(1976.0),
 np.float64(1977.0),
 np.float64(1978.0),
 np.float64(1979.0),
 np.float64(1980.0),
 np.float64(1981.0),
 np.float64(1982.0),
 np.float64(1983.0),
 np.float64(1984.0),
 np.float64(1985.0),
 np.float64(1986.0),
 np.float64(1987.0),
 np.float64(1988.0),
 np.float64(1989.0),
 np.float64(1990.0),
 np.float64(1991.0),
 np.float64(1992.0),
 np.float64(1993.0),
 np.float64(1994.0),
 np.float64(1995.0),
 np.float64(1996.0),
 np.float64(1997.0),
 np.float64(1998.0),
 np.float64(1999.0),
 np.float64(2000.0),
 np.float64(2001.0),
 np.float64(2002.0),
 np.float64(2003.0),
 np.float64(2004.0),
 np.float64(2005.0),
 np.float64(2006.0),
 np.float64(2007.0),
 np.float64(2008.0),
 np.float64(2009.0),
 np.float64(2010.0),
 np.float64(2011.0),
 np.float64(2012.0),
 np.float64(2013.0),
 np.float64(2014.0),
 np.float64(nan)]

In [ ]:
computacao_resp2014['ANO_FIM_EM'].value_counts()

,count
ANO_FIM_EM,
2009.0,1203
2008.0,1018
2010.0,936
2007.0,873
2006.0,625
2005.0,585
2004.0,495
2003.0,380
2011.0,326


In [ ]:
sorted(computacao_resp2014['ANO_IN_GRAD'].unique())

[np.float64(1991.0),
 np.float64(1995.0),
 np.float64(1996.0),
 np.float64(1997.0),
 np.float64(1998.0),
 np.float64(1999.0),
 np.float64(2000.0),
 np.float64(2001.0),
 np.float64(2002.0),
 np.float64(2003.0),
 np.float64(2004.0),
 np.float64(2005.0),
 np.float64(2006.0),
 np.float64(2007.0),
 np.float64(2008.0),
 np.float64(2009.0),
 np.float64(2010.0),
 np.float64(2011.0),
 np.float64(2012.0),
 np.float64(2013.0),
 np.float64(2014.0),
 np.float64(nan)]

In [ ]:
computacao_resp2014['ANO_IN_GRAD'].value_counts()

,count
ANO_IN_GRAD,
2011.0,1934
2010.0,1781
2012.0,1276
2009.0,1184
2008.0,628
2013.0,472
2007.0,386
2006.0,225
2005.0,114


### 2017

In [ ]:
# 2017
processar_enade_por_curso(
    base_filtrada='/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2017_filtrado.csv',
    pasta_ano='2017',
    edicao_enade=2017,
    lista_numeros_arquivos=[1,2, 3, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 23, 27],
    lista_atributos=[
        "NU_ANO", "CO_CURSO", "CO_GRUPO", "CO_UF_CURSO", "CO_REGIAO_CURSO",
        "CO_CATEGAD", "CO_ORGACAD", "CO_TURNO_GRADUACAO", "TP_PRES",
        'NT_GER', 'TP_SEXO', 'NU_IDADE', 'ANO_FIM_EM', 'ANO_IN_GRAD',
        'QE_I01', 'QE_I02', 'QE_I03',
        'QE_I04', 'QE_I05', 'QE_I08', 'QE_I09', 'QE_I10', 'QE_I11',
        'QE_I12', 'QE_I13', 'QE_I15', 'QE_I17', 'QE_I21'
    ]
)

✅ Arquivo salvo com sucesso: /content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2017/computacao_enade_2017.csv


In [ ]:
computacao_resp2017 = pd.read_csv('/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2017/computacao_enade_2017.csv', sep=';')
computacao_resp2017.shape

(7129, 28)

### 2021

In [ ]:
# 2017
processar_enade_por_curso(
    base_filtrada='/content/drive/MyDrive/EDM 2025.1/ENADE/enade_2021_filtrado.csv',
    pasta_ano='2021',
    edicao_enade=2021,
    lista_numeros_arquivos=[1,2, 3, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 23, 27],
    lista_atributos=[
        "NU_ANO", "CO_CURSO", "CO_GRUPO", "CO_UF_CURSO", "CO_REGIAO_CURSO",
        "CO_CATEGAD", "CO_ORGACAD", "CO_TURNO_GRADUACAO", "TP_PRES",
        'NT_GER', 'TP_SEXO', 'NU_IDADE', 'ANO_FIM_EM', 'ANO_IN_GRAD',
        'QE_I01', 'QE_I02', 'QE_I03',
        'QE_I04', 'QE_I05', 'QE_I08', 'QE_I09', 'QE_I10', 'QE_I11',
        'QE_I12', 'QE_I13', 'QE_I15', 'QE_I17', 'QE_I21'
    ]
)

✅ Arquivo salvo com sucesso: /content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2021/computacao_enade_2021.csv


In [ ]:
computacao_resp2021 = pd.read_csv('/content/drive/MyDrive/EDM 2025.1/DATASETS/ENADE/2021/computacao_enade_2021.csv', sep=';')
computacao_resp2021.shape

(7564, 28)

In [ ]:
outliers_afem = computacao_resp2021[(computacao_resp2021['ANO_FIM_EM'] < 1955) | (computacao_resp2021['ANO_FIM_EM'] > 2014)]
outliers_afem

,NU_ANO,CO_CURSO,CO_GRUPO,CO_UF_CURSO,CO_REGIAO_CURSO,CO_CATEGAD,CO_ORGACAD,CO_TURNO_GRADUACAO,TP_PRES,NT_GER,...,QE_I05,QE_I08,QE_I09,QE_I10,QE_I11,QE_I12,QE_I13,QE_I15,QE_I17,QE_I21
0,2021,317,4004,28,2,1,10028,2.0,555,43.4,...,C,C,E,NaN,A,NaN,D,E,A,A
1,2021,317,4004,28,2,1,10028,2.0,555,42.4,...,C,E,C,NaN,A,NaN,B,A,A,A
2,2021,317,4004,28,2,1,10028,2.0,555,34.8,...,E,C,NaN,B,A,NaN,A,E,A,A
3,2021,317,4004,28,2,1,10028,2.0,555,34.7,...,A,B,NaN,A,A,NaN,A,A,D,A
6,2021,317,4004,28,2,1,10028,2.0,555,52.4,...,D,E,NaN,A,A,A,A,E,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7545,2021,5001459,72,26,2,5,10022,4.0,555,40.2,...,E,C,B,E,B,A,A,A,A,A
7551,2021,5001459,72,26,2,5,10022,4.0,555,56.6,...,E,B,E,A,B,A,A,A,B,A
7555,2021,5001459,72,26,2,5,10022,4.0,555,41.2,...,D,F,C,E,B,A,A,A,B,A
7558,2021,5001459,72,26,2,5,10022,4.0,555,31.8,...,D,C,C,A,B,A,A,A,A,A


In [ ]:
outliers_aig = computacao_resp2021[(computacao_resp2021['ANO_IN_GRAD'] < 1977) | (computacao_resp2021['ANO_IN_GRAD'] > 2014)]
outliers_aig

,NU_ANO,CO_CURSO,CO_GRUPO,CO_UF_CURSO,CO_REGIAO_CURSO,CO_CATEGAD,CO_ORGACAD,CO_TURNO_GRADUACAO,TP_PRES,NT_GER,...,QE_I05,QE_I08,QE_I09,QE_I10,QE_I11,QE_I12,QE_I13,QE_I15,QE_I17,QE_I21
0,2021,317,4004,28,2,1,10028,2.0,555,43.4,...,C,C,E,NaN,A,NaN,D,E,A,A
1,2021,317,4004,28,2,1,10028,2.0,555,42.4,...,C,E,C,NaN,A,NaN,B,A,A,A
2,2021,317,4004,28,2,1,10028,2.0,555,34.8,...,E,C,NaN,B,A,NaN,A,E,A,A
3,2021,317,4004,28,2,1,10028,2.0,555,34.7,...,A,B,NaN,A,A,NaN,A,A,D,A
6,2021,317,4004,28,2,1,10028,2.0,555,52.4,...,D,E,NaN,A,A,A,A,E,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7559,2021,5001459,72,26,2,5,10022,4.0,555,28.6,...,D,F,C,A,B,A,A,A,B,B
7560,2021,5001459,72,26,2,5,10022,4.0,555,12.7,...,D,A,C,E,I,A,F,A,B,B
7561,2021,5001459,72,26,2,5,10022,4.0,555,25.4,...,D,C,C,A,I,A,A,A,A,A
7562,2021,5001459,72,26,2,5,10022,4.0,334,NaN,...,F,A,B,A,I,A,A,A,B,A


In [ ]:
sorted(computacao_resp2021['ANO_FIM_EM'].unique())

[np.float64(1905.0),
 np.float64(1972.0),
 np.float64(1976.0),
 np.float64(1977.0),
 np.float64(1978.0),
 np.float64(1979.0),
 np.float64(1980.0),
 np.float64(1981.0),
 np.float64(1982.0),
 np.float64(1983.0),
 np.float64(1984.0),
 np.float64(1986.0),
 np.float64(1987.0),
 np.float64(1988.0),
 np.float64(1989.0),
 np.float64(1990.0),
 np.float64(1991.0),
 np.float64(1992.0),
 np.float64(1993.0),
 np.float64(1994.0),
 np.float64(1995.0),
 np.float64(1996.0),
 np.float64(1997.0),
 np.float64(1998.0),
 np.float64(1999.0),
 np.float64(2000.0),
 np.float64(2001.0),
 np.float64(2002.0),
 np.float64(2003.0),
 np.float64(2004.0),
 np.float64(2005.0),
 np.float64(2006.0),
 np.float64(2007.0),
 np.float64(2008.0),
 np.float64(2009.0),
 np.float64(2010.0),
 np.float64(2011.0),
 np.float64(2012.0),
 np.float64(2013.0),
 np.float64(2014.0),
 np.float64(2015.0),
 np.float64(2016.0),
 np.float64(2017.0),
 np.float64(2018.0),
 np.float64(2019.0),
 np.float64(2020.0),
 np.float64(nan),
 np.float64(2021

In [ ]:
computacao_resp2021['ANO_FIM_EM'].value_counts()

,count
ANO_FIM_EM,
2016.0,970
2015.0,953
2014.0,845
2013.0,681
2017.0,631
2012.0,510
2011.0,403
2010.0,359
2018.0,293


In [ ]:
sorted(computacao_resp2021['ANO_IN_GRAD'].unique())

[np.float64(219.0),
 np.float64(1993.0),
 np.float64(1995.0),
 np.float64(1996.0),
 np.float64(1999.0),
 np.float64(2001.0),
 np.float64(2002.0),
 np.float64(2003.0),
 np.float64(2004.0),
 np.float64(2005.0),
 np.float64(2006.0),
 np.float64(2007.0),
 np.float64(2008.0),
 np.float64(2009.0),
 np.float64(2010.0),
 np.float64(2011.0),
 np.float64(2012.0),
 np.float64(2013.0),
 np.float64(2014.0),
 np.float64(2015.0),
 np.float64(2016.0),
 np.float64(2017.0),
 np.float64(2018.0),
 np.float64(2019.0),
 np.float64(2020.0),
 np.float64(2021.0),
 np.float64(nan)]

In [ ]:
computacao_resp2014['ANO_IN_GRAD'].value_counts()

,count
ANO_IN_GRAD,
2011.0,1934
2010.0,1781
2012.0,1276
2009.0,1184
2008.0,628
2013.0,472
2007.0,386
2006.0,225
2005.0,114
